# Change Detection

In [ ]:
import geoai
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from pathlib import Path
from geoai.change_detection import ChangeDetection
import geopandas as gpd
from IPython.display import Image

from libs.sentinel import get_sentinel_image_thumbnail
from libs.ee import initialize_ee
ee = initialize_ee(get_ipython())

from libs.s2cloudless import S2Cloudless

In [ ]:
directory = 'data/NZ_landslide_inventory/'
auckland_filepath = f'{directory}/cleaned_auckland.gpkg'
gdf = gpd.read_file(auckland_filepath, layer='landslides')
gdf = gdf[gdf['dateoccurence'].notnull()]
gdf

In [ ]:
auckland_dir = './data/2-data-explore-events'

event = gdf.loc[0]

landslide_img = f'{auckland_dir}/marks/0_174.701880487593_-36.931306466746_2023-02-14_2023-04-15.png'

display(event)
display(Image(filename=landslide_img))

## Find before landslide images

In [ ]:
start_date = (
        event['dateoccurence'].date() - pd.Timedelta(days=60)
    ).strftime("%Y-%m-%d")
end_date = event['dateoccurence'].date().strftime("%Y-%m-%d")

start_date, end_date

In [ ]:
s2cloudless = S2Cloudless(ee)

point = event['geometry']
# Extract coordinates from Shapely Point
lon = point.x
lat = point.y

# Create Earth Engine Point geometry
aoi = ee.Geometry.Point([lon, lat]).buffer(1000)

# start_date
# end_date,
buffer=50
cloud_filter=30

m_cloud_free = s2cloudless.cloud_free_map(aoi, start_date, end_date, buffer=buffer, cloud_filter=cloud_filter)
m_cloud_free.addLayer(ee.Geometry.Point([lon, lat]), {'color': 'red'}, 'My Pinpoint')
m_cloud_free

## Export all `tif` file

In [ ]:
out_dir = './data/4_change_detection/'
start_date_offset = -90
end_date_offset = -6 # s2 revisit time is 5 days, make sure we have images before the event

for idx, event in gdf.iterrows():
    filepath_rgb, filepath_with_marker = get_sentinel_image_thumbnail(
        ee,
        s2cloudless,
        event,
        idx,
        out_dir,
        start_date_offset=start_date_offset,
        end_date_offset=end_date_offset,
        buffer=buffer,
        cloud_filter=cloud_filter,
    )

    display(f"Filename {filepath_rgb}")
    display(Image(filename=filepath_with_marker))